In [1]:
CATALOGUE = "https://catalogue.dataspace.copernicus.eu/stac"

In [2]:
import requests
import json
from typing import Any
from info import PASSWORD

In [42]:
def list_collections() -> list[str]:
    collection_info = json.loads(requests.get(f"{CATALOGUE}/collections").content)
    return [x['id'] for x in collection_info['collections']]

def get_collection(collection_id: str) -> dict[str, Any]:
    info = json.loads(requests.get(f"{CATALOGUE}/collections/{collection_id}").content)
    return info

def list_items(collection_id: str, bbox = None, datetime_from=None) -> list:
    addition = ''
    if bbox:
        flattened = [x for xs in bbox for x in xs]
        flattened = [str(x) for x in flattened]
        new_part = ','.join(flattened)
        new_part = 'bbox=' + new_part + '&'
        addition += new_part
    if datetime_from is not None:
        time_str = f"{datetime_from.isoformat()}/"
        new_part = f"datetime={time_str}&"
        addition += new_part
    string = f"{CATALOGUE}/collections/{collection_id}/items?{addition}limit=50&sortby=+datetime"
    info = json.loads(requests.get(string).content)
    return info['features']

In [43]:
list_collections()

['COP-DEM',
 'S2GLC',
 'TERRAAQUA',
 'SENTINEL-3',
 'SENTINEL-5P',
 'SENTINEL-1-RTC',
 'SENTINEL-1',
 'SMOS',
 'LANDSAT-7',
 'CCM',
 'LANDSAT-5',
 'LANDSAT-8',
 'ENVISAT',
 'SENTINEL-6',
 'GLOBAL-MOSAICS',
 'SENTINEL-2']



The approximate conversions are:

    Latitude: 1 deg = 110.574 km
    Longitude: 1 deg = 111.320*cos(latitude) km


In [44]:
import math

In [45]:
LATID_CHANGE = 1/110.574

def long_change(latitude: float) -> float:
    return 1 / 111.320 / math.cos(math.radians(latitude))
def create_bounding_box(latitude, longitude, width) -> tuple[tuple[float, float]]:
    half = width / 2.0
    delta_latitude = half * LATID_CHANGE
    delta_longitude = half * long_change(latitude)

    return [(latitude + delta_latitude, longitude - delta_longitude),
            (latitude + delta_latitude, longitude + delta_longitude),
            (latitude - delta_latitude, longitude + delta_longitude),
            (latitude - delta_latitude, longitude - delta_longitude)]

In [46]:
center = (50.1232, 14.538) # prague smh
width = 10
box = create_bounding_box(*center, width)
box

[(50.16841858664785, 14.46794411061371),
 (50.16841858664785, 14.608055889386291),
 (50.07798141335214, 14.608055889386291),
 (50.07798141335214, 14.46794411061371)]

In [47]:
LOGIN = 'kirschales@gmail.com'
def authenticate(login: str, password: str) -> bytes | str:
    data = {
        'grant_type': 'password',
        'username': login,
        'password': password,
        'client_id': 'cdse-public'
    }
    return json.loads(requests.post('https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token', data).content)

In [48]:
access_token = authenticate(LOGIN, PASSWORD)

In [49]:
name = "S5P_NRTI_L2__AER_LH_20240914T100517_20240914T101017_35866_03_020701_20240914T104552.nc"

In [50]:
json.loads(requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Name eq '{name}'").content)['value']

[{'@odata.mediaContentType': 'application/octet-stream',
  'Id': 'fdd7b8c5-add8-40b7-bf60-4ce4e4bef3f0',
  'Name': 'S5P_NRTI_L2__AER_LH_20240914T100517_20240914T101017_35866_03_020701_20240914T104552.nc',
  'ContentType': 'application/octet-stream',
  'ContentLength': 16339653,
  'OriginDate': '2024-09-14T11:00:08.967000Z',
  'PublicationDate': '2024-09-14T11:05:46.838147Z',
  'ModificationDate': '2024-09-14T11:05:48.545175Z',
  'Online': True,
  'EvictionDate': '9999-12-31T23:59:59.999999Z',
  'S3Path': '/eodata/Sentinel-5P/TROPOMI/L2__AER_LH/2024/09/14/S5P_NRTI_L2__AER_LH_20240914T100517_20240914T101017_35866_03_020701_20240914T104552',
  'Checksum': [{'Value': 'f91c93512ad3759f749dd74bf62783ec',
    'Algorithm': 'MD5',
    'ChecksumDate': '2024-09-14T11:05:48.152266Z'},
   {'Value': '5142c1f1ba6561dea8f7231e1099dd9d3bc9dc97ecff60757c6926588328077c',
    'Algorithm': 'BLAKE3',
    'ChecksumDate': '2024-09-14T11:05:48.209087Z'}],
  'ContentDate': {'Start': '2024-09-14T10:05:11.000000Z

In [51]:
my_id = json.loads(requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Name eq '{name}'").content)['value'][0]['Id']

In [52]:
url = f"https://download.dataspace.copernicus.eu/odata/v1/Products({my_id})/$value"

headers = {"Authorization": f"Bearer {access_token}"}

# Create a session and update headers
session = requests.Session()
session.headers.update(headers)

# Perform the GET request
response = session.get(url, stream=True)

# Check if the request was successful
if response.status_code == 200:
    with open("product.zip", "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:  # filter out keep-alive new chunks
                file.write(chunk)
else:
    print(f"Failed to download file. Status code: {response.status_code}")
    print(response.text)

Failed to download file. Status code: 401
{"detail":"Unauthorized"}


In [61]:

import datetime
from shapely import Polygon
import pandas as pd
def contains_bbox(bbox, geometry):
    bbox = Polygon(bbox)
    geometry = Polygon(geometry)
    return geometry.contains(bbox)

def not_too_long(x):
    start = x['properties']['start_datetime']
    end = x['properties']['end_datetime']
    start = datetime.datetime.fromisoformat(start)
    end = datetime.datetime.fromisoformat(end)
    return end - start < datetime.timedelta(hours=1)
def get_new_updates(latitude: float,
                    longitude: float,
                    time_from: datetime.datetime,
                    collection: str = "SENTINEL-5P",
                    width: float | int = 10):
    box = create_bounding_box(latitude, longitude, width)
    items = list_items(collection, bbox=[box[0], box[2]], datetime_from=time_from)
    items = [x for x in items if not_too_long(x)]
    [x for x in items if contains_bbox(box, x['geometry']['coordinates'][0])]
    datetimes = pd.DataFrame(data={'datetime': [datetime.datetime.fromisoformat(x['properties']['datetime']) for x in items]})
    datetimes['date'] = datetimes['datetime'].dt.date
    return list(datetimes.groupby(by=['date']).mean()['datetime'])
    
    
    

In [66]:
def get_all_updates(positions: list[tuple], time_from: datetime.datetime, collection: str = "SENTINEL-5P", width: float = 10) -> dict[tuple, list[datetime.datetime]]:
    all_updates = {}
    for position in positions:
        new_updates = get_new_updates(
            position[0],
            position[1],
            time_from,
            collection,
            width
        )
        if len(new_updates):
            all_updates[position] = new_updates
    return all_updates

In [62]:
bruh = datetime.datetime(year=2021, month=5, day=8)
items = get_new_updates(*center, bruh)
len(items)

2

In [63]:
items

[Timestamp('2021-05-08 09:12:52.979591936+0000', tz='UTC'),
 Timestamp('2021-05-09 08:50:01+0000', tz='UTC')]